# Matching precincts

In [2]:
import pandas
import geopandas
import matplotlib.pyplot as plt
from fuzzywuzzy import process

from create_absentee_ballots_csv import create_absentee_ballots_df
from match_precincts import find_matches

In [2]:
df = create_absentee_ballots_df()
gdf = geopandas.read_file("./shapefiles/GA_precincts16.shp")

In [3]:
counties = df["COUNTY"].unique()

for county in counties:
    difference = len(df[df["COUNTY"] == county]) - len(gdf[gdf["CTYNAME"] == county])
    if difference > 0:
        print(county, difference)

Bibb 1
Chattooga 1
Columbia 3
Crawford 3
DeKalb 3
Effingham 2
Fayette 2
Forsyth 4
Fulton 1
Gwinnett 1
Henry 1
Lowndes 2
Muscogee 2
Newton 2
Pickens 1


In [4]:
matches = find_matches(df, gdf)

In [5]:
average_by_county = matches.groupby("county").rating.mean()
average_by_county[average_by_county < 90]

county
Bleckley          0.000000
Chatham          89.101124
Chattahoochee    86.000000
Clarke           76.500000
Crawford         54.555556
Rockdale         82.777778
Spalding          0.000000
Taylor           56.000000
Name: rating, dtype: float64

In [6]:
bad_counties = set(average_by_county[average_by_county < 90].index)

### Spalding

In [35]:
spalding_df = df[df["COUNTY"] == "Spalding"]
spalding_gdf = gdf[gdf["CTYNAME"] == "Spalding"]

In [38]:
manual_spalding_matches = {}

for i, votes in spalding_df["PRES16D"].items():
    for j, other_votes in spalding_gdf["PRES16D"].items():
        if votes == int(other_votes):
            manual_spalding_matches[spalding_df["PRECINCT"][i]] = spalding_gdf["PRECINCT_N"][j]

In [39]:
manual_spalding_matches

{'SECOND BAPTISH CH': '17',
 'FAIRMONT': '10',
 'SENIOR CENTER': '18',
 'THIRD WARD': '20',
 'UGA CAMPUS': '21',
 'CITY PARK': '06',
 'FIRST METHODIST': '12',
 'BOY SCOUTS HDQTRS': '03',
 'CABIN': '04',
 'CARVER': '05',
 'FIRST PRESBYTERIA': '13',
 'LIBERTY': '15',
 'COUNTY LINE': '08',
 'SUNNYSIDE METHODI': '19',
 'BLALOCK': '02',
 'GARY REID': '14',
 'AMBUCS': '01',
 'COMMUNITY': '07',
 'FIRST ASSEMBLY': '11',
 'ROBERTS': '16',
 'EAGES LANDING BAPTIST': '09'}

In [476]:
assert_mapping(spalding_df["PRECINCT"], spalding_gdf["PRECINCT_N"], manual_spalding_matches)

In [477]:
final_spalding = merge_precincts_by_mapping(spalding_df, manual_spalding_matches, "Spalding")

In [479]:
final_spalding

,PRECINCT,PRES16D,PRES16D_AB,PRES16D_AD,PRES16D_ED,PRES16D_PR,PRES16L,PRES16L_AB,PRES16L_AD,PRES16L_ED,PRES16L_PR,PRES16R,PRES16R_AB,PRES16R_AD,PRES16R_ED,PRES16R_PR,REG_VOTE,COUNTY
0,01,347,11,200,136,0,16,0,7,9,0,672,22,381,268,1,1529,Spalding
1,02,44,0,16,28,0,12,0,4,8,0,368,6,105,257,0,501,Spalding
2,03,267,10,154,103,0,25,4,12,9,0,867,28,600,239,0,1432,Spalding
3,04,151,3,101,47,0,25,1,12,12,0,927,30,505,391,1,1419,Spalding
4,05,380,11,203,164,2,27,1,16,10,0,796,32,511,253,0,1500,Spalding
5,06,839,36,447,356,0,15,0,11,4,0,221,18,141,62,0,1762,Spalding
6,07,759,29,410,320,0,16,0,8,8,0,123,5,78,40,0,1340,Spalding
7,08,171,7,88,75,1,41,2,10,29,0,1261,30,643,587,1,1875,Spalding
8,09,931,64,640,227,0,56,1,34,21,0,1720,79,1183,457,1,3208,Spalding
9,10,855,57,414,384,0,11,0,5,6,0,72,3,46,23,0,1588,Spalding


In [478]:
final_spalding.to_csv("./merged/spalding.csv", index=False)

### Rockdale

In [24]:
print(set(df[df["COUNTY"] == "Rockdale"]["PRECINCT"]))
print(set(gdf[gdf["CTYNAME"] == "Rockdale"]["PRECINCT_N"]))

{'Olde Towne', 'Flat Shoals', 'Conyers', 'Smyrna', 'Honey Creek', 'Magnet', 'Bethel', 'Rockdale', 'Sheffield', 'Barksdale', 'Milstead', 'St. Pius', 'Fieldstone', 'The Lakes', 'Stanton', 'Salem', 'Hightower', 'Lorraine'}
{'CO', 'SA', 'FS', 'HI', 'SH', 'SP', 'BT', 'BA', 'MI', 'RO', 'OT', 'HC', 'SM', 'ST', 'FI', 'LO', 'LA', 'MA'}


In [25]:
manual_rockdale_matches = {
    "Olde Towne": "OT",
    "Flat Shoals": "FS",
    "Conyers": "CO",
    "Smyrna": "SM",
    "Honey Creek": "HC",
    "Magnet": "MA",
    "Bethel": "BT",
    "Rockdale": "RO",
    "Sheffield": "SH",
    "Barksdale": "BA",
    "Milstead": "MI",
    "St. Pius": "SP",
    "Fieldstone": "FI",
    "The Lakes": "LA",
    "Stanton": "ST",
    "Salem": "SA",
    "Hightower": "HI",
    "Lorraine": "LO"
}

In [26]:
key_set = set(manual_rockdale_matches.keys())
value_set = set(manual_rockdale_matches.values())
assert len(manual_rockdale_matches) == len(key_set)
assert len(manual_rockdale_matches) == len(value_set)

In [27]:
assert set(df[df["COUNTY"] == "Rockdale"]["PRECINCT"]) == key_set
assert set(gdf[gdf["CTYNAME"] == "Rockdale"]["PRECINCT_N"]) == value_set

In [403]:
rockdale_df, rockdale_gdf = get_county_df_and_gdf("Rockdale")
merged_rockdale = merge_precincts_by_mapping(rockdale_df, manual_rockdale_matches, "Rockdale")

In [404]:
assert len(merged_rockdale) == len(rockdale_gdf)

In [406]:
merged_rockdale.to_csv("./merged/rockdale.csv", index=False)

### Clarke

In [387]:
clarke_matches = find_matches(df[df["COUNTY"] == "Clarke"], gdf[gdf["CTYNAME"] == "Clarke"], column="PRECINCT_I")

In [388]:
clarke_matches

,county,index,match_index,precinct,rating,match_value
0,Clarke,401,2545,1A W'ville Train Depot,60,1A
1,Clarke,402,2543,1B Gaines School,90,1B
2,Clarke,403,2537,1C Whit Davis School,60,1C
3,Clarke,404,2542,1D Whit Davis School,60,1D
4,Clarke,405,2561,2A Judia J. Harris Sch,60,2A
5,Clarke,406,2544,2B Howard Stroud School,60,2B
6,Clarke,407,2611,3A Clarke Central High,60,3A
7,Clarke,408,2560,3B Thomas N. Lay Park,60,3B
8,Clarke,409,2540,4A Athens Transit Ctr,60,4A
9,Clarke,410,2557,4B Memorial Park,90,4B


OK, those look right.

In [389]:
clarke_mapping = {
    df["PRECINCT"][i]: gdf["PRECINCT_N"][j]
    for i, j in zip(clarke_matches["index"], clarke_matches["match_index"])
}

In [393]:
clarke_df, clarke_gdf = get_county_df_and_gdf("Clarke")

In [394]:
assert_mapping(clarke_df["PRECINCT"], clarke_gdf["PRECINCT_N"], clarke_mapping)

In [395]:
joined_clarke = merge_precincts_by_mapping(clarke_df, clarke_mapping, "Clarke")

In [396]:
joined_clarke.to_csv("./merged/clarke.csv", index=False)

### Taylor

These matches actually look OK, thanks to the numbers:

In [7]:
matches[matches["county"] == "Taylor"]

,county,index,match_index,precinct,rating,match_value
1148,Taylor,2455,516,#1 BUTLER FIRE DEPT.,86,PRECINCT 1
1149,Taylor,2456,515,#5 COURTHOUSE,34,PRECINCT 5
1150,Taylor,2457,517,#6 REYNOLDS,48,PRECINCT 6


### Crawford

In [376]:
crawford = df[df["COUNTY"] == "Crawford"]
crawford_gdf = gdf[gdf["CTYNAME"] == "Crawford"]

In [377]:
crawford_gdf.columns

Index(['ID', 'POPULATION', 'DISTRICT', 'PRECINCT_I', 'PRECINCT_N', 'CTYNAME',
       'FIPS1', 'FIPS2', 'PRES16D', 'PRES16R', 'PRES16L', 'SEN16D', 'SEN16R',
       'SEN16L', 'TOTPOP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN',
       'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'HISP', 'H_WHITE', 'H_BLACK',
       'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP',
       'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP',
       '2MOREVAP', 'CONDIST', 'HDIST', 'SENDIST', 'geometry'],
      dtype='object')

In [378]:
crawford.columns

Index(['COUNTY', 'PRECINCT', 'PRES16D', 'PRES16D_AB', 'PRES16D_AD',
       'PRES16D_ED', 'PRES16D_PR', 'PRES16L', 'PRES16L_AB', 'PRES16L_AD',
       'PRES16L_ED', 'PRES16L_PR', 'PRES16R', 'PRES16R_AB', 'PRES16R_AD',
       'PRES16R_ED', 'PRES16R_PR', 'REG_VOTE'],
      dtype='object')

In [379]:
columns_to_aggregate = [
    'PRES16D', 'PRES16D_AB', 'PRES16D_AD',
    'PRES16D_ED', 'PRES16D_PR', 'PRES16L', 'PRES16L_AB', 'PRES16L_AD',
    'PRES16L_ED', 'PRES16L_PR', 'PRES16R', 'PRES16R_AB', 'PRES16R_AD',
    'PRES16R_ED', 'PRES16R_PR', 'REG_VOTE'
]

It looks like the shapefile has merged the "COUNTY" and "CITY" parts of each of precincts 1B, 2, and 3 into singular precincts 1B, 2, and 3.

In [380]:
merged_crawford_precincts = crawford.groupby(crawford["PRECINCT"].apply(lambda s: s.split()[0])).sum()
merged_crawford_precincts

,PRES16D,PRES16D_AB,PRES16D_AD,PRES16D_ED,PRES16D_PR,PRES16L,PRES16L_AB,PRES16L_AD,PRES16L_ED,PRES16L_PR,PRES16R,PRES16R_AB,PRES16R_AD,PRES16R_ED,PRES16R_PR,REG_VOTE
PRECINCT,,,,,,,,,,,,,,,,
1A,135,9,45,81,0,3,0,1,2,0,154,3,66,84,1,377
1B,389,29,152,207,1,10,1,4,5,0,225,8,125,92,0,813
2,507,32,334,141,0,24,0,16,8,0,548,39,377,132,0,1385
3,154,7,87,59,1,31,3,9,19,0,875,36,514,325,0,1331
4,126,11,61,54,0,16,0,7,9,0,906,36,398,471,1,1318
5,110,7,47,56,0,22,0,11,11,0,927,31,384,511,1,1301


In [381]:
merged_crawford_precincts.reset_index(inplace=True)
merged_crawford_precincts["COUNTY"] = "Crawford"

In [382]:
assert set(merged_crawford_precincts.columns) == set(df.columns)

In [385]:
merged_crawford_precincts["PRECINCT"] = merged_crawford_precincts["PRECINCT"].apply(lambda s: f"DISTRICT {s}")

In [386]:
merged_crawford_precincts.to_csv("./merged/crawford.csv", index=False)

### Bleckley

In [30]:
df[df["COUNTY"] == "Bleckley"]

,COUNTY,PRECINCT,PRES16D,PRES16D_AB,PRES16D_AD,PRES16D_ED,PRES16D_PR,PRES16L,PRES16L_AB,PRES16L_AD,PRES16L_ED,PRES16L_PR,PRES16R,PRES16R_AB,PRES16R_AD,PRES16R_ED,PRES16R_PR,REG_VOTE
127,Bleckley,FAIRGROUND,1101,113,502,479,7,123,1,66,56,0,3719,136,2076,1505,2,6162


In [31]:
gdf[gdf["CTYNAME"] == "Bleckley"]

,ID,POPULATION,DISTRICT,PRECINCT_I,PRECINCT_N,CTYNAME,FIPS1,FIPS2,PRES16D,PRES16R,...,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,CONDIST,HDIST,SENDIST,geometry
878,1754047,13063,0231,1,1,Bleckley,13023,023,1101,3719,...,2655,7,87,3,0,49,008,144,020,"POLYGON ((-83.337441 32.530466, -83.337288 32...."


This one is just a single precinct in both datasets, and the vote totals match, so I feel good about matching it.

### Spalding

In [471]:
df[df["COUNTY"] == "Spalding"]

,COUNTY,PRECINCT,PRES16D,PRES16D_AB,PRES16D_AD,PRES16D_ED,PRES16D_PR,PRES16L,PRES16L_AB,PRES16L_AD,PRES16L_ED,PRES16L_PR,PRES16R,PRES16R_AB,PRES16R_AD,PRES16R_ED,PRES16R_PR,REG_VOTE
2401,Spalding,SECOND BAPTISH CH,741,27,336,376,2,8,0,3,5,0,101,5,59,37,0,1582
2402,Spalding,FAIRMONT,855,57,414,384,0,11,0,5,6,0,72,3,46,23,0,1588
2403,Spalding,SENIOR CENTER,192,20,120,52,0,39,3,22,14,0,740,39,544,157,0,1254
2404,Spalding,THIRD WARD,180,10,100,70,0,38,1,14,23,0,634,33,397,204,0,1083
2405,Spalding,UGA CAMPUS,975,24,528,422,1,56,2,34,20,0,831,46,589,196,0,2939
2406,Spalding,CITY PARK,839,36,447,356,0,15,0,11,4,0,221,18,141,62,0,1762
2407,Spalding,FIRST METHODIST,227,12,139,76,0,42,3,10,29,0,883,34,588,261,0,1377
2408,Spalding,BOY SCOUTS HDQTRS,267,10,154,103,0,25,4,12,9,0,867,28,600,239,0,1432
2409,Spalding,CABIN,151,3,101,47,0,25,1,12,12,0,927,30,505,391,1,1419
2410,Spalding,CARVER,380,11,203,164,2,27,1,16,10,0,796,32,511,253,0,1500


In [472]:
gdf[gdf["CTYNAME"] == "Spalding"]

,ID,POPULATION,DISTRICT,PRECINCT_I,PRECINCT_N,CTYNAME,FIPS1,FIPS2,PRES16D,PRES16R,...,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,CONDIST,HDIST,SENDIST,geometry
165,1054638,3210,25514,14,19,Spalding,13255,255,235,722,...,316,7,8,0,3,8,003,073,016,"POLYGON ((-84.247136 33.338347, -84.247125 33...."
595,1089620,2656,25509,09,04,Spalding,13255,255,151,927,...,132,5,10,0,1,24,003,130,016,"POLYGON ((-84.08899099999999 33.285646, -84.08..."
598,816003,829,25515,15,02,Spalding,13255,255,44,368,...,9,3,2,0,0,2,003,073,016,"POLYGON ((-84.50851299999999 33.239846, -84.50..."
610,989093,2143,25512,12,15,Spalding,13255,255,177,766,...,160,3,7,0,0,11,003,073,016,"POLYGON ((-84.437512 33.208678, -84.442915 33...."
611,989165,3714,25519,19,11,Spalding,13255,255,405,1248,...,426,11,26,0,1,23,003,073,016,"(POLYGON ((-84.419723 33.289229, -84.419655000..."
654,1015373,1822,25503,03,18,Spalding,13255,255,192,740,...,106,2,13,0,0,9,003,130,016,"POLYGON ((-84.244934 33.229792, -84.2449309999..."
655,1015495,3276,25502,02,10,Spalding,13255,255,855,72,...,2000,7,3,4,0,17,003,130,016,"POLYGON ((-84.243694 33.248529, -84.243719 33...."
656,1015584,2322,25518,18,07,Spalding,13255,255,759,123,...,1263,6,0,5,0,21,003,073,016,"POLYGON ((-84.253989 33.303513, -84.247848 33...."
657,1015655,3019,25517,17,01,Spalding,13255,255,347,672,...,508,6,3,3,0,25,003,130,016,"(POLYGON ((-84.22183200000001 33.234187, -84.2..."
658,1063865,3170,25521,21,09,Spalding,13255,255,931,1720,...,399,6,9,1,1,18,003,130,016,"POLYGON ((-84.136798 33.308318, -84.1366419999..."


This one is wild! I suppose our next course of action is try and figure out where these landmarks ("SENIOR CENTER", "CITY PARK") are and then match that to our precinct geometries. We could also talk to the county for more information.

### Bacon

In [34]:
df[df["COUNTY"] == "Bacon"]

,COUNTY,PRECINCT,PRES16D,PRES16D_AB,PRES16D_AD,PRES16D_ED,PRES16D_PR,PRES16L,PRES16L_AB,PRES16L_AD,PRES16L_ED,PRES16L_PR,PRES16R,PRES16R_AB,PRES16R_AD,PRES16R_ED,PRES16R_PR,REG_VOTE
18,Bacon,Pearson County,136,10,66,59,1,6,0,1,5,0,850,25,550,271,4,1339
19,Bacon,Axson,29,2,12,15,0,4,0,1,3,0,375,10,144,221,0,561
20,Bacon,Willacoochee,257,4,74,179,0,16,1,3,12,0,489,14,191,283,1,1064
21,Bacon,Pearson City,275,14,147,114,0,9,0,5,4,0,164,9,87,68,0,731


In [35]:
gdf[gdf["CTYNAME"] == "Bacon"]

,ID,POPULATION,DISTRICT,PRECINCT_I,PRECINCT_N,CTYNAME,FIPS1,FIPS2,PRES16D,PRES16R,...,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,CONDIST,HDIST,SENDIST,geometry
728,2300131,1978,005NEW L,NEW L,NEW LACY,Bacon,13005,005,58,696,...,109,3,1,1,0,7,001,169,007,"POLYGON ((-82.31152299999999 31.590997, -82.31..."
787,1443845,848,005WARE,WARE,WARE,Bacon,13005,005,25,328,...,17,0,1,0,0,4,001,169,007,"POLYGON ((-82.455321 31.491416, -82.453571 31...."
1082,2292290,1483,005TAYLO,TAYLO,TAYLOR,Bacon,13005,005,30,521,...,31,0,3,0,0,7,001,169,007,"POLYGON ((-82.595156 31.53239, -82.59652199999..."
1083,2292423,520,005WARNO,WARNO,WARNOCK,Bacon,13005,005,18,241,...,6,0,2,0,0,5,001,169,007,"POLYGON ((-82.40184000000001 31.63957, -82.401..."
1091,2285630,6267,005DOUGL,DOUGL,DOUGLAS,Bacon,13005,005,477,1578,...,1022,4,20,5,1,26,001,169,007,"POLYGON ((-82.433916 31.504452, -82.434225 31...."


### Bibb

In [417]:
merge_by_auto_matching("Bibb", 90).to_csv('./merged/bibb.csv', index=False)

### Henry

In [426]:
merge_by_auto_matching("Henry", 80).to_csv("./merged/henry.csv", index=False)

### Pickens

In [420]:
merge_by_auto_matching("Pickens", 90).to_csv("./merged/pickens.csv", index=False)

In [131]:
pickens = df[df["COUNTY"] == "Pickens"]
pickens_gdf = gdf[gdf["CTYNAME"] == "Pickens"]

In [132]:
columns_to_aggregate = [
    'PRES16D', 'PRES16D_AB', 'PRES16D_AD',
    'PRES16D_ED', 'PRES16D_PR', 'PRES16L', 'PRES16L_AB', 'PRES16L_AD',
    'PRES16L_ED', 'PRES16L_PR', 'PRES16R', 'PRES16R_AB', 'PRES16R_AD',
    'PRES16R_ED', 'PRES16R_PR', 'REG_VOTE'
]

In [ ]:
pickens # 13

In [ ]:
pickens_gdf #12

In [137]:
merged_pickens_precincts = pickens.loc[[2252,2253]].sum()
merged_pickens_precincts['COUNTY']= 'Pickens'
merged_pickens_precincts['PRECINCT']= 'TATE'
merged_pickens_precincts

COUNTY        Pickens
PRECINCT         TATE
PRES16D           174
PRES16D_AB          6
PRES16D_AD        104
PRES16D_ED         63
PRES16D_PR          1
PRES16L            42
PRES16L_AB          1
PRES16L_AD         29
PRES16L_ED         12
PRES16L_PR          0
PRES16R          1358
PRES16R_AB         12
PRES16R_AD        822
PRES16R_ED        523
PRES16R_PR          1
REG_VOTE         1992
dtype: object

In [139]:
pickens_dropped=pickens.drop([2252,2253])

In [145]:
pickens_dropped.append(merged_pickens_precincts,ignore_index=True)

,COUNTY,PRECINCT,PRES16D,PRES16D_AB,PRES16D_AD,PRES16D_ED,PRES16D_PR,PRES16L,PRES16L_AB,PRES16L_AD,PRES16L_ED,PRES16L_PR,PRES16R,PRES16R_AB,PRES16R_AD,PRES16R_ED,PRES16R_PR,REG_VOTE
0,Pickens,YELLOW CREEK,365,14,230,120,1,36,2,16,18,0,1725,43,1014,668,0,2562
1,Pickens,SHARPTOP,340,15,241,84,0,46,2,22,22,0,1434,39,926,469,0,2120
2,Pickens,APPALACHIAN,167,7,115,45,0,33,2,14,17,0,970,15,689,266,0,1463
3,Pickens,REFUGE,169,3,119,47,0,29,0,13,16,0,1259,25,838,396,0,1825
4,Pickens,HILL,158,5,91,62,0,42,1,15,26,0,913,13,576,324,0,1385
5,Pickens,HINTON,47,2,25,20,0,16,0,6,10,0,404,1,167,236,0,585
6,Pickens,JERUSALEM,67,1,33,33,0,12,2,3,7,0,538,6,193,339,0,776
7,Pickens,LUDVILLE,24,0,12,12,0,10,0,1,9,0,343,2,130,211,0,474
8,Pickens,NELSON,118,0,54,64,0,25,0,4,21,0,750,11,404,335,0,1098
9,Pickens,TALKING ROCK,104,5,58,41,0,19,0,16,3,0,933,14,553,366,0,1328


In [146]:
pickens_dropped.to_csv('./merged/pickens.csv',index=False)

### Chattooga

In [42]:
chattooga = df[df["COUNTY"] == "Chattooga"]
chattooga_gdf = gdf[gdf["CTYNAME"] == "Chattooga"]

In [43]:
len(chattooga), len(chattooga_gdf)

,COUNTY,PRECINCT,PRES16D,PRES16D_AB,PRES16D_AD,PRES16D_ED,PRES16D_PR,PRES16L,PRES16L_AB,PRES16L_AD,PRES16L_ED,PRES16L_PR,PRES16R,PRES16R_AB,PRES16R_AD,PRES16R_ED,PRES16R_PR,REG_VOTE
346,Chattooga,CLOUDLAND,58,3,30,25,0,5,0,3,2,0,158,9,43,106,0,275
347,Chattooga,PENNVILLE,48,3,21,24,0,3,0,1,2,0,272,8,139,125,0,488
348,Chattooga,COLDWATER,53,1,10,42,0,3,0,1,2,0,106,3,51,52,0,203
349,Chattooga,DIRTSELLER,16,0,3,13,0,0,0,0,0,0,106,3,42,61,0,156
350,Chattooga,HAYWOOD,15,2,1,12,0,3,0,1,2,0,89,0,15,74,0,133
351,Chattooga,LYERLY,94,4,34,56,0,12,0,2,10,0,445,7,186,252,0,740
352,Chattooga,TRION,306,14,176,116,0,36,1,12,23,0,1589,25,861,702,1,2686
353,Chattooga,SUMMERVILLE,759,40,453,265,1,67,4,32,31,0,2099,59,1390,648,2,4046
354,Chattooga,TELOGA,40,3,14,23,0,10,0,8,2,0,320,5,128,187,0,464
355,Chattooga,DIRTTOWN,96,8,35,53,0,11,0,2,9,0,455,7,166,280,2,690


In [50]:
chattooga.loc[[346, 354]]

,COUNTY,PRECINCT,PRES16D,PRES16D_AB,PRES16D_AD,PRES16D_ED,PRES16D_PR,PRES16L,PRES16L_AB,PRES16L_AD,PRES16L_ED,PRES16L_PR,PRES16R,PRES16R_AB,PRES16R_AD,PRES16R_ED,PRES16R_PR,REG_VOTE
346,Chattooga,CLOUDLAND,58,3,30,25,0,5,0,3,2,0,158,9,43,106,0,275
354,Chattooga,TELOGA,40,3,14,23,0,10,0,8,2,0,320,5,128,187,0,464


In [48]:
chattooga_gdf.loc[[1519]]

,ID,POPULATION,DISTRICT,PRECINCT_I,PRECINCT_N,CTYNAME,FIPS1,FIPS2,PRES16D,PRES16R,...,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,CONDIST,HDIST,SENDIST,geometry
1519,3588199,1385,055XC-968,XC-968,CLOULDLAND-TELOGA,Chattooga,13055,55,98,478,...,4,9,2,0,4,6,014,012,053,"POLYGON ((-85.40821099999999 34.532039, -85.40..."


In [53]:
name = "CLOUDLAND-TELOGA"
def find_cloudland_and_teloga(precinct):
    if precinct in name:
        return name
    else:
        return precinct

In [58]:
merged_chattooga = chattooga.groupby(chattooga["PRECINCT"].apply(find_cloudland_and_teloga)).sum().reset_index()

In [61]:
merged_chattooga["COUNTY"] = "Chattooga"

In [349]:
merged_chattooga.to_csv("./merged/chattooga.csv", index=False)

## Fixing Morgan

In [3]:
morgan_df = df[df["COUNTY"] == "Morgan"]
morgan_gdf = gdf[gdf["CTYNAME"] == "Morgan"]

In [4]:
morgan_match = find_matches(morgan_df, morgan_gdf)

In [12]:
mapping = morgan_match.set_index("match_index")["index"]

In [15]:
cols = ["PRES16D", "PRES16R", "PRES16L"]
patch_rows = pandas.DataFrame({col: mapping.map(morgan_df[col]) for col in cols})

In [17]:
gdf.update(patch_rows)

In [20]:
gdf.to_file("./shapefiles/GA_precincts16.shp")

## Effingham

In [465]:
effingham_final = merge_by_auto_matching("Effingham", 90)
effingham_final.to_csv("./merged/effingham.csv", index=False)

In [466]:
effingham_df, effingham_gdf = get_county_df_and_gdf("Effingham")

### Columbia

In [69]:
def get_county_df_and_gdf(county, df=df, gdf=gdf):
    return df[df["COUNTY"] == county], gdf[gdf["CTYNAME"] == county]

In [110]:
def get_good_matches(df, gdf, threshold=100):
    matches = find_matches(df, gdf)
    good_matches = matches[matches["rating"] >= threshold]
    mapping = dict(zip(good_matches["precinct"], good_matches["match_value"]))
    return mapping

In [206]:
def assert_mapping(left, right, mapping):
    expected_keys = set(left)
    keys = set(mapping.keys())
    missing_keys = expected_keys - keys
    extra_keys = keys - expected_keys
    assert len(missing_keys) == 0, f"Missing expected keys: {missing_keys}"
    assert len(extra_keys) == 0, f"Found unexpected keys: {extra_keys}"
    values = set(mapping.values())
    expected_values = set(right)
    missing_values = expected_values - values
    extra_values = values - expected_values
    assert len(missing_values) == 0, f"Missing expected values: {missing_values}"
    assert len(extra_values) == 0, f"Found unexpected values: {extra_values}"


In [70]:
columbia_df, columbia_gdf = get_county_df_and_gdf("Columbia")

In [73]:
columbia_matches = find_matches(columbia_df, columbia_gdf)

In [80]:
columbia_matches

,county,index,match_index,precinct,rating,match_value
0,Columbia,662,193,Kiokee Baptist Church-10,86,GENESIS CHURCH
1,Columbia,663,193,Kiokee Baptist Church-12,86,GENESIS CHURCH
2,Columbia,664,209,Lewis Methodist Church,95,LEWIS METHODIST
3,Columbia,665,229,Woodlawn Baptist Church,93,WOODLAWN BAPT CHURCH
4,Columbia,666,228,Patriots Park,100,PATRIOTS PARK
5,Columbia,667,1105,Harlem Middle School-10,95,HARLEM MIDDLE SCHOOL
6,Columbia,668,1105,Harlem Middle School-12,95,HARLEM MIDDLE SCHOOL
7,Columbia,669,1104,Harlem Baptist Church-10,90,HARLEM BAPTIST
8,Columbia,670,1104,Harlem Baptist Church-12,90,HARLEM BAPTIST
9,Columbia,671,947,Bessie Thomas Center,100,BESSIE THOMAS CENTER


In [137]:
columbia_mapping = get_good_matches(columbia_df, columbia_gdf, 90)

In [138]:
columbia_mapping["Kiokee Baptist Church-10"] = "KIOKEE BAPT CHURCH"
columbia_mapping["Kiokee Baptist Church-12"] = "KIOKEE BAPT CHURCH"

In [207]:
assert_mapping(columbia_df["PRECINCT"], columbia_gdf["PRECINCT_N"], columbia_mapping)

In [ ]:
merged_columbia = columbia_df.groupby(columbia_df["PRECINCT"].apply(columbia_mapping.get)).sum()

merged_columbia = merged_columbia.reset_index(drop=False)

merged_columbia["COUNTY"] = "Columbia"

In [350]:
merged_columbia.to_csv("./merged/columbia.csv", index=False)

### DeKalb

In [101]:
dekalb_df, dekalb_gdf = get_county_df_and_gdf("DeKalb")

In [119]:
matches = find_matches(dekalb_df, dekalb_gdf)

In [129]:
matches[matches["rating"] < 90]

,county,index,match_index,precinct,rating,match_value
52,DeKalb,832,78,EAST LAKE ELEM,86,CANDLER-MURPHEY CANDLER ELEM
67,DeKalb,847,2358,GEORGETOWN SQUARE,80,GEORGETOWN SQ (DUN)
69,DeKalb,849,2274,GLENNWOOD - 04,82,GLENNWOOD (DEC)
70,DeKalb,850,2274,GLENNWOOD - 05,82,GLENNWOOD (DEC)
118,DeKalb,898,2404,MT. VERNON EAST,84,MOUNT VERNON EAST (DUN)
119,DeKalb,899,2360,MT. VERNON WEST,84,MOUNT VERNON WEST (DUN)
135,DeKalb,915,78,PLEASANTDALE ELEM,86,CANDLER-MURPHEY CANDLER ELEM
161,DeKalb,941,2255,SNAPFINGER ROAD NORTH,89,SNAPFINGER ROAD S
162,DeKalb,942,2255,SNAPFINGER ROAD SOUTH,89,SNAPFINGER ROAD S
168,DeKalb,948,2105,STONE MOUNTAIN CHAMPION,86,STONE MILL ELEM


In [148]:
dekalb_mapping = get_good_matches(dekalb_df, dekalb_gdf, 0)

In [156]:
manual_matches = {
    'LITHONIA': 'LITHONIA (LIT)',
    'BROCKETT': 'BROCKETT (TUC)',
    'WOODWARD ELEM': 'WOODWARD (BHAVN)',
    'COVINGTON HWY LIBRARY': 'COVINGTON HWY',
    "DUNWOODY": 'DUNWOODY (DUN)',
    "EAST LAKE ELEM": 'EAST LAKE (ATL)',
    "PLEASANTDALE ELEM": "PLEASANTDALE ROAD",
    "SNAPFINGER ROAD NORTH": "SNAPFINGER ROAD N",
    "SNAPFINGER ROAD SOUTH": "SNAPFINGER ROAD S",
    "STONE MOUNTAIN LIBRARY": 'STONE MTN LIBRARY  (STO)',
    "STONE MOUNTAIN CHAMPION": 'STONE MTN CHAMPION (STO)',
    "STONE MOUNTAIN MIDDLE": 'STN MTN MIDDLE',
}

In [157]:
dekalb_mapping.update(manual_matches)

In [159]:
assert_mapping(dekalb_df["PRECINCT"], dekalb_gdf["PRECINCT_N"], dekalb_mapping)

In [164]:
left = dekalb_df.set_index("PRECINCT")["PRES16D"]
right = dekalb_gdf.set_index("PRECINCT_N")["PRES16D"].astype(int)

for key, value in dekalb_mapping.items():
    if left[key] != right[value]:
        print(key, left[key], "|", value, right[value])

AVONDALE HIGH - 04 954 | AVONDALE HIGH 1804
AVONDALE HIGH - 05 850 | AVONDALE HIGH 1804
GLENNWOOD - 04 559 | GLENNWOOD (DEC) 1278
GLENNWOOD - 05 719 | GLENNWOOD (DEC) 1278
MCNAIR ACADEMY 1131 | MCNAIR ACADEMY 2032
MCNAIR HIGH 2032 | MCNAIR HIGH 1225
MCNAIR MIDDLE 1225 | MCNAIR 1131
MILLER GROVE HIGH 1450 | MILLER GROVE HIGH 2013
MILLER GROVE MIDDLE 2013 | MILLER GROVE 1450
WADSWORTH ELEM - 05 486 | WADSWORTH 1543
WADSWORTH ELEM - 04 1057 | WADSWORTH 1543


Looks like the shapefile has MCNAIR ACADEMY, MCNAIR HIGH, and MCNAIR MIDDLE shuffled, as well as MILLER GROVE HIGH and MILLER GROVE MIDDLE. Hmm... I guess these will be fixed when we join the dataset, since we'll bring the vote totals from the tabular data.

In [165]:
def merge_precincts_by_mapping(county_df, mapping, county):
    merged = county_df.groupby(county_df["PRECINCT"].apply(mapping.get)).sum()
    merged= merged.reset_index(drop=False)
    merged["COUNTY"] = county
    return merged

In [166]:
merged_dekalb = merge_precincts_by_mapping(dekalb_df, dekalb_mapping, "DeKalb")

In [167]:
assert len(merged_dekalb) == len(dekalb_gdf)

In [358]:
merged_dekalb.to_csv("./merged/dekalb.csv", index=False)

### Chatham

In [232]:
chatham_df, chatham_gdf = get_county_df_and_gdf("Chatham")
chatham_df = chatham_df.copy()

In [233]:
chatham_df["PRECINCT"] = chatham_df["PRECINCT"].apply(lambda s: s.split()[0])

matches = find_matches(chatham_df, chatham_gdf, "PRECINCT_I")

chatham_mapping = {df["PRECINCT"][i]: gdf["PRECINCT_N"][j] for i, j in zip(matches["index"], matches["match_index"])}

In [234]:
assert_mapping(df[df["COUNTY"] == "Chatham"]["PRECINCT"], chatham_gdf["PRECINCT_N"], chatham_mapping)

In [357]:
final_chatham = df[df["COUNTY"] == "Chatham"].copy()

final_chatham["PRECINCT"] = final_chatham["PRECINCT"].apply(chatham_mapping.get)

final_chatham.to_csv("./merged/chatham.csv", index=False)

In [239]:
assert len(final_chatham) == len(chatham_gdf)

### Forsyth

In [222]:
forsyth_df, forsyth_gdf = get_county_df_and_gdf("Forsyth")

In [226]:
set(forsyth_df["PRECINCT"].apply(lambda s: s.split()[0])) == set(forsyth_gdf["PRECINCT_I"])

True

Looks like we can just use the prefixes to match, as above.

In [236]:
forsyth_df = forsyth_df.copy()

forsyth_df["PRECINCT"] = forsyth_df["PRECINCT"].apply(lambda s: s.split()[0])

matches = find_matches(forsyth_df, forsyth_gdf, "PRECINCT_I")

forsyth_mapping = {df["PRECINCT"][i]: gdf["PRECINCT_N"][j] for i, j in zip(matches["index"], matches["match_index"])}

In [237]:
assert_mapping(df[df["COUNTY"] == "Forsyth"]["PRECINCT"], forsyth_gdf["PRECINCT_N"], forsyth_mapping)

In [243]:
forsyth_df = df[df["COUNTY"] == "Forsyth"].copy()
final_forsyth = forsyth_df.groupby(forsyth_df["PRECINCT"].apply(forsyth_mapping.get)).sum()
final_forsyth.reset_index(drop=False, inplace=True)
final_forsyth["COUNTY"] = "Forsyth"

In [245]:
assert set(final_forsyth["PRECINCT"]) == set(forsyth_gdf["PRECINCT_N"])
assert len(final_forsyth) == len(forsyth_gdf)

In [356]:
final_forsyth.to_csv("./merged/forsyth.csv", index=False)

### Fayette

In [248]:
fayette_df, fayette_gdf = get_county_df_and_gdf("Fayette")

In [256]:
fayette_mapping = get_good_matches(fayette_df, fayette_gdf, 95)

In [260]:
final_fayette = fayette_df.groupby(fayette_df["PRECINCT"].apply(fayette_mapping.get)).sum()
final_fayette.reset_index(drop=False, inplace=True)
final_fayette["COUNTY"] = "Fayette"

In [261]:
assert set(final_fayette["PRECINCT"]) == set(fayette_gdf["PRECINCT_N"])
assert len(final_fayette) == len(fayette_gdf)

In [355]:
final_fayette.to_csv("./merged/fayette.csv", index=False)

### Fulton

In [281]:
def merge_by_auto_matching(county, threshold=90):
    df, gdf = get_county_df_and_gdf(county)
    mapping = get_good_matches(df, gdf, threshold)
    assert_mapping(df["PRECINCT"], gdf["PRECINCT_N"], mapping)
    final = merge_precincts_by_mapping(df, mapping, county)
    assert set(final["PRECINCT"]) == set(gdf["PRECINCT_N"])
    assert len(final) == len(gdf)
    return final

In [282]:
final_fulton = merge_by_auto_matching("Fulton", threshold=90)

In [354]:
final_fulton.to_csv("./merged/fulton.csv", index=False)

### Gwinnett

In [291]:
gwinnett_df, gwinnett_gdf = get_county_df_and_gdf("Gwinnett")

In [292]:
auto_matches = find_matches(gwinnett_df, gwinnett_gdf)

In [295]:
auto_matches[auto_matches.rating < 100]

,county,index,match_index,precinct,rating,match_value
19,Gwinnett,1637,2378,PINCKNEYVILLE A,97,PINCKNEYVILLE A1
35,Gwinnett,1653,2083,035 Cates D - 04,90,CATES D


In [300]:
gwinnett_matches = get_good_matches(gwinnett_df, gwinnett_gdf, 100)

In [305]:
gwinnett_matches["PINCKNEYVILLE A"] = "PINKCNEYVILLE A"
gwinnett_matches["035 Cates D - 04"] = "CATES D"

In [306]:
assert_mapping(gwinnett_df["PRECINCT"], gwinnett_gdf["PRECINCT_N"], gwinnett_matches)

In [307]:
final_gwinnett = merge_precincts_by_mapping(gwinnett_df, gwinnett_matches, "Gwinnett")

In [352]:
final_gwinnett.to_csv("./merged/gwinnett.csv", index=False)

### Lowndes

In [353]:
final_lowndes = merge_by_auto_matching("Lowndes", 90)
final_lowndes.to_csv("./merged/lowndes.csv", index=False)

### Muscogee

In [316]:
muscogee_df, muscogee_gdf = get_county_df_and_gdf("Muscogee")

In [323]:
muscogee_matches = get_good_matches(muscogee_df, muscogee_gdf, 95)

In [324]:
manual_matches = {
    "EPWORTH-03": "EPWORTH UMC",
    "EPWORTH-02": "EPWORTH UMC",
    'ST PAUL U.M.C.-02': "ST PAUL/CLUBVIEW",
    'ST PAUL U.M.C.-03': "ST PAUL/CLUBVIEW",
    'ST PETER U. M. C.': "ST. PETER",
    'BRITT DAVID BAPT CH': "BRITT DAVID",
    "CORNERSTONE CHURCH": "CORNERSTONE",
    'GALLOPS/HANNAN': "MARIANNA GALLOPS",
    'WYNNTON': "WYNNTON/BRITT",
}

muscogee_matches.update(manual_matches)

In [325]:
assert_mapping(muscogee_df["PRECINCT"], muscogee_gdf["PRECINCT_N"], muscogee_matches)

In [326]:
final_muscogee = merge_precincts_by_mapping(muscogee_df, muscogee_matches, "Muscogee")

In [327]:
assert len(final_muscogee) == len(muscogee_gdf)

In [336]:
final_muscogee.set_index("PRECINCT").sort_index()["PRES16D"] == muscogee_gdf.set_index("PRECINCT_N").sort_index()["PRES16D"].astype(int)

PRECINCT
BRITT DAVID                 True
CARVER/MACK                 True
COLUMBUS TECH               True
CORNERSTONE                 True
CUSSETA RD                  True
EDDY/KEY                    True
EDGEWOOD BAPTIST            True
EPWORTH UMC                 True
FAITH TABERNACLE            True
FIRST AFRICAN               True
FORT/WADDELL                True
GENTIAN/REESE @LDS          True
MARIANNA GALLOPS            True
MOON/MORNINGSIDE            True
MT PILGRIM                  True
NATIONAL INFANTRY MUSEUM    True
PSALMOND/MATHEWS            True
ROTHSCHILD                  True
SALVATION ARMY              True
ST ANDREWS/MIDLAND          True
ST JOHN/BELVEDERE           True
ST MARK/HEIFERHORN          True
ST PAUL/CLUBVIEW            True
ST. PETER                   True
WYNNBROOK                   True
WYNNTON/BRITT               True
Name: PRES16D, dtype: bool

In [351]:
final_muscogee.to_csv("./merged/muscogee.csv", index=False)

### Newton

In [342]:
final_newton = merge_by_auto_matching("Newton", 90)

In [343]:
final_newton.set_index("PRECINCT").sort_index()["PRES16D"] == gdf[gdf["CTYNAME"] == "Newton"].set_index("PRECINCT_N").sort_index()["PRES16D"].astype(int)

PRECINCT
ALCOVY             True
ALMON              True
BEAVERDAM          True
BREWERS            True
BRICK STORE        True
BUCK CREEK         True
CEDAR SHOALS       True
CITY POND          True
COVINGTON MILLS    True
CROWELL            True
DOWNS              True
FAIRVIEW           True
GUM CREEK          True
HUB                True
LEGUINN            True
LIVINGSTON         True
MANSFIELD          True
NEWBORN            True
OXFORD             True
ROCKY PLAINS       True
STANSELLS          True
TOWN               True
Name: PRES16D, dtype: bool

In [347]:
final_newton.to_csv("./merged/newton.csv", index=False)

### Laurens

In [434]:
laurens_df, laurens_gdf = get_county_df_and_gdf("Laurens")

In [436]:
laurens_matches = get_good_matches(laurens_df, laurens_gdf, 100)

In [438]:
set(laurens_df["PRECINCT"]) - set(laurens_matches.keys())

{'Family Life Center', 'LCFS 10 (Valambrosia)'}

In [439]:
set(laurens_gdf["PRECINCT_N"]) - set(laurens_matches.values())

{'FBC - FLC', 'LCFS #10 (VALAMBROSIA)'}

In [440]:
laurens_matches["Family Life Center"] = "FBC - FLC"
laurens_matches["LCFS 10 (Valambrosia)"] = "LCFS #10 (VALAMBROSIA)"

In [441]:
assert_mapping(laurens_df["PRECINCT"], laurens_gdf["PRECINCT_N"], laurens_matches)

In [443]:
laurens_final = merge_precincts_by_mapping(laurens_df, laurens_matches, "Laurens")

In [444]:
laurens_final.to_csv("./merged/laurens.csv", index=False)

### Chattahoochee and Bleckley

In [447]:
merge_by_auto_matching("Chattahoochee", 0).to_csv("./merged/chattahoochee.csv", index=False)

In [448]:
merge_by_auto_matching("Bleckley", 0).to_csv("./merged/bleckley.csv", index=False)

### Jackson

In [449]:
jackson_df, jackson_gdf = get_county_df_and_gdf("Jackson")

In [454]:
jackson_matches = get_good_matches(jackson_df, jackson_gdf, 100)

In [457]:
jackson_matches["South Jefferson"] = "S JEFFERSON"
jackson_matches["North Jefferson"] = "N JEFFERSON"
jackson_matches["South Minish"] = "S MINISH"
jackson_matches["North Minish"] = "N MINISH"

In [458]:
assert_mapping(jackson_df["PRECINCT"], jackson_gdf["PRECINCT_N"], jackson_matches)

In [459]:
merge_precincts_by_mapping(jackson_df, jackson_matches, "Jackson").to_csv("./merged/jackson.csv", index=False)